In [10]:
import tensorflow as tf
import numpy as np

rnd = np.random.RandomState(seed=13)
n_units = 10
n_in = 3
n_out = 1

x = tf.placeholder(dtype=tf.float32, shape=(None, n_in), name="ExternalInput")
W_init = rnd.normal(size=(n_in, n_units), loc=0, scale=.01)
b_init = np.zeros(shape=(n_units,))
W = tf.Variable(initial_value=W_init, name='W', dtype=tf.float32)
b = tf.Variable(initial_value=b_init, name='b', dtype=tf.float32)
a = tf.matmul(x, W) + b
h = tf.tanh(a)
W_out_init = rnd.normal(size=(n_units, n_out), loc=0, scale=.01)
W_out = tf.Variable(initial_value=W_out_init, name='W_out', dtype=tf.float32)

b_out_init = np.zeros(shape=(n_out,))
b_out = tf.Variable(initial_value=b_out_init, name='b_out', dtype=tf.float32)
y = tf.matmul(h, W_out) + b_out

In [11]:
# task
t = tf.reduce_sum(x, axis=1)/3
loss = tf.reduce_mean(tf.abs((y-t)), axis=0)

In [12]:
# training step
alpha = 0.001
gW_out, gb_out = tf.gradients(loss, [W_out, b_out])
gh = tf.gradients(loss, h)[0]
h_1 = h - alpha * gh

# a_star = a  # TODO
cutoff = 0.99


def arctanh(x):
    return (tf.log(1 + x) - tf.log(1 - x)) / 2

a_star = tf.where(tf.logical_and(h_1 > h, h > cutoff), a,
                  tf.where(tf.logical_and(h_1 < h, h < -cutoff), a, arctanh(h_1)))

d = a_star - a

ga_W_in, ga_b_in = tf.gradients(tf.reduce_mean((a - a_star) ** 2, axis=0), [W, b])
# ga_W_in, ga_b_in = tf.gradients(loss, [W, b])

grad_var_pairs = ((gW_out, W_out), (gb_out, b_out), (ga_b_in, b), (ga_W_in, W))
grads = [tf.reshape(g[0], [-1]) for g in grad_var_pairs]
gradient = tf.concat(0, grads)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha)
train_op = optimizer.apply_gradients(grad_var_pairs)

In [13]:
import sys
sys.stdout = open('/dev/stdout', 'w')

x_val = rnd.uniform(size=(2000, n_in))
batch_size = 50

sess = tf.InteractiveSession()
sess.run(tf.local_variables_initializer())
sess.run(tf.global_variables_initializer())

print("Training...")
for i in range(1000):
    x_np = rnd.uniform(size=(batch_size, n_in), low=0, high=1)
    _, grad_norm = sess.run((train_op, tf.reduce_sum(tf.reduce_sum(gradient**2))), feed_dict={x: x_np})
    if i % 100 == 0:
        val_error = sess.run(tf.reduce_mean(tf.reduce_mean(tf.abs(y - t))), feed_dict={x: x_val})
        print('Iteration {} -> val error:{:.2f}, g_norm: {:.2f}'.format(i, val_error, grad_norm))
        sys.stdout.flush()

In [9]:
x_np = rnd.uniform(size=(3, n_in))
print("x: ", x_np)
y_, t_, loss_ = sess.run((y, t, loss), feed_dict={x: x_np})
print("y: ", y_)
print("t: ", t_)
print("loss: ", loss_)